# Replicate GluNet on T1DEXI

GluNet was mainly reported as a personalized model

In [1]:
from __future__ import division, print_function

import collections
import csv
import datetime
import xml.etree.ElementTree as ET

import numpy as np
import pandas as pd

from datetime import datetime, timedelta
from scipy.interpolate import CubicSpline
import matplotlib.pyplot as plt
import os

from datetime import datetime, timedelta
from scipy.interpolate import CubicSpline
import matplotlib.pyplot as plt

import torch
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [2]:
ph = 6
history_len = 15

In [3]:
def round_up_to_nearest_five_minutes(ts):
    # Parse the timestamp
    dt = datetime.strptime(ts, "%d-%m-%Y %H:%M:%S")
    
    # Calculate minutes to add to round up to the nearest 5 minutes
    minutes_to_add = (5 - dt.minute % 5) % 5
    if minutes_to_add == 0 and dt.second == 0:
        # If exactly on a 5 minute mark and second is 0, no need to add time
        minutes_to_add = 5
    
    # Add the necessary minutes
    new_dt = dt + timedelta(minutes=minutes_to_add)
    
    # Return the new timestamp in the same format
    return new_dt.strftime( "%d-%m-%Y %H:%M:%S")


def preprocess_t1dexi_cgm(path, round):

    subject = pd.read_csv(path)
    # Group by 'Category' column
    # grouped = subject.groupby('LBCAT')
    # Create a dictionary to store the split DataFrames
    # split_dfs = {category: group for category, group in grouped}
    # selected_cgm = split_dfs["CGM"][["LBORRES", "LBDTC"]]
    # new_df_cgm = pd.DataFrame(selected_cgm)
    new_df_cgm = subject[["LBORRES", "LBDTC"]]

    new_df_cgm['LBDTC'] = pd.to_datetime(new_df_cgm['LBDTC'], errors='coerce')  # Convert 'date' column to datetime if not already
    new_df_cgm.sort_values('LBDTC', inplace=True)  # Sort the DataFrame by the 'date' column

    if round == True:
        rounded_timestamp = []
        for ts in new_df_cgm["LBDTC"]:
            rounded_timestamp.append(round_up_to_nearest_five_minutes(ts))
        new_df_cgm["rounded_LBDTC"] = rounded_timestamp
        formatted_data = [[{'ts': row['rounded_LBDTC'], 'value': row['LBORRES']}] for _, row in new_df_cgm.iterrows()]

    else:
        # Convert each row to the desired format
        formatted_data = [[{'ts': row['LBDTC'].to_pydatetime(), 'value': row['LBORRES']}] for _, row in new_df_cgm.iterrows()]
    
    return formatted_data

In [4]:
def segement_data_as_1hour(data):
    df = pd.DataFrame(data)

    # Calculate time differences
    df['time_diff'] = df['timestamp'].diff()

    # Identify large gaps
    df['new_segment'] = df['time_diff'] > pd.Timedelta(hours=1)

    # Find indices where new segments start
    segment_starts = df[df['new_segment']].index

    # Initialize an empty dictionary to store segments
    segments = {}
    prev_index = 0

    # Loop through each segment start and slice the DataFrame accordingly
    for i, start in enumerate(segment_starts, 1):
        segments[f'segment_{i}'] = df.iloc[prev_index:start].reset_index(drop=True)
        prev_index = start

    # Add the last segment from the last gap to the end of the DataFrame
    segments[f'segment_{len(segment_starts) + 1}'] = df.iloc[prev_index:].reset_index(drop=True)

    # Optionally remove helper columns from each segment
    for segment in segments.values():
        segment.drop(columns=['time_diff', 'new_segment'], inplace=True)
    
    return segments

In [5]:
def detect_missing_and_spline_interpolate(segments):
    for sequence in segments:
        # sequence = "segment_3"
        detected_missing = 0
        for ts in range(len( segments[sequence]['timestamp'])-1):
            if segments[sequence]['timestamp'][ts+1] - segments[sequence]['timestamp'][ts] > timedelta(minutes = 6):
                print(sequence)
                print("before: ", segments[sequence]['timestamp'][ts])
                print("after: ", segments[sequence]['timestamp'][ts+1])
                detected_missing = 1
            
        if detected_missing == 1:
            datetime_list = np.array(pd.date_range(start=min(segments[sequence]['timestamp']), end=max(segments[sequence]['timestamp']), freq='5T').tolist())
            reference_time = min(segments[sequence]['timestamp'])

            # Convert datetime objects to the number of seconds since the reference time
            datetime_seconds_since_start = [((dt - reference_time).total_seconds())/60 for dt in datetime_list] # Make it into minute
            original_timestamp_in_segement = [((dt - reference_time).total_seconds())/60 for dt in segments[sequence]['timestamp']]

            x = original_timestamp_in_segement
            y = np.array(segments[sequence]['glucose_value'])
            cs = CubicSpline(x, y)
            xs = datetime_seconds_since_start

            interpolated_xs = cs(xs)
            time_index_interpolated = pd.date_range(start=reference_time, periods=len(interpolated_xs), freq='5T')

            # Create DataFrame from the time index and glucose values
            df_interpolated = pd.DataFrame({'timestamp': time_index_interpolated, 'glucose_value': interpolated_xs})
            segments[sequence] = df_interpolated

    return segments

In [6]:
# Function to align and update segments with meal data
def update_segments_with_meals(segments, meal_df):
    for segment_name, segment_df in segments.items():
        # Initialize the 'carbs' column to zeros
        segment_df['carbs'] = 0

        # Iterate through each timestamp in the segment
        for i, row in segment_df.iterrows():
            # Find the closest meal timestamp and its carb information
            meal_df['time_difference'] = abs(meal_df['ts'] - row['timestamp'])
            closest_meal = meal_df.loc[meal_df['time_difference'].idxmin()]
            
            # Check if the closest meal is within 5 minutes
            if closest_meal['time_difference'] <= pd.Timedelta(minutes=5):
                # Ensure that the meal is assigned to only one segment and is the closest
                if not meal_df.at[closest_meal.name, 'assigned']:
                    segment_df.at[i, 'carbs'] = closest_meal['carbs']
                    meal_df.at[closest_meal.name, 'assigned'] = True  # Mark as assigned
                else:
                    # Check if the current timestamp is closer than the one it was assigned to
                    assigned_index = segment_df[segment_df['carbs'] == closest_meal['carbs']].index[0]
                    if row['timestamp'] - closest_meal['ts'] < segment_df.at[assigned_index, 'timestamp'] - closest_meal['ts']:
                        # Reassign the meal to the new closer timestamp
                        segment_df.at[assigned_index, 'carbs'] = 0  # Remove carbs from previously assigned timestamp
                        segment_df.at[i, 'carbs'] = closest_meal['carbs']  # Assign carbs to the new closer timestamp
            # else:
            #     print(f"Meal type {meal['type']} on {meal['ts']} is too far from closest timestamp in {closest_segment} with a difference of {closest_diff}.")

    return segments

In [7]:
# Function to align and update segments with meal data
def update_segments_with_basal(segments, basal_df):
    for segment_name, segment_df in segments.items():
        # Initialize the 'carbs' column to zeros
        segment_df['basal_rate'] = None

        # Iterate through each timestamp in the segment
        for i, row in segment_df.iterrows():
            # Find the closest meal timestamp and its carb information
            for _, basal_row in basal_df.iterrows():
                if basal_row['ts'] <= row['timestamp'] < (basal_row['end_ts'] if pd.notna(basal_row['end_ts']) else pd.Timestamp('2099-12-31')):
                    segment_df.at[i, 'basal_rate'] = basal_row['value']
                    break

    return segments

In [8]:
# Read in bolus and temp basal information
# Need to set the 
def preprocess_t1dexi_bolus_tempbasal(filepath, round):
    subject_facm = pd.read_csv(filepath)
    # Group by 'Category' column
    grouped = subject_facm.groupby('FACAT')

    split_dfs = {category: group for category, group in grouped}
    # Step 1: Extract the desired columns
    new_df_bolus = split_dfs["BOLUS"][["FAORRES", "FADTC"]]
    new_df_bolus['FADTC'] = pd.to_datetime(new_df_bolus['FADTC'], format="%Y-%m-%d %H:%M:%S")
    new_df_bolus.reset_index(drop=True, inplace=True)
    new_df_bolus = new_df_bolus.rename(columns={'FAORRES': 'dose', 'FADTC': 'ts_begin'})
    new_df_bolus['assigned'] = False
    # new_df_bolus['end_ts'] = new_df_bolus['ts_begin'].shift(-1)
    return new_df_bolus

In [9]:
def update_segments_with_bolus(segments, bolus_df):
    for segment_name, segment_df in segments.items():
        # Initialize the 'dose' column to zeros
        segment_df['bolus_dose'] = 0

        # Iterate through each timestamp in the segment
        for i, row in segment_df.iterrows():
            # Find the closest bolus timestamp and its carb information
            bolus_df['time_difference'] = abs(bolus_df['ts_begin'] - row['timestamp'])
            closest_bolus = bolus_df.loc[bolus_df['time_difference'].idxmin()]
            
            # Check if the closest bolus is within 5 minutes
            if closest_bolus['time_difference'] <= pd.Timedelta(minutes=5):
                # Ensure that the bolus is assigned to only one segment and is the closest
                if not bolus_df.at[closest_bolus.name, 'assigned']:
                    segment_df.at[i, 'bolus_dose'] = closest_bolus['dose']
                    bolus_df.at[closest_bolus.name, 'assigned'] = True  # Mark as assigned
                else:
                    # Check if the current timestamp is closer than the one it was assigned to
                    assigned_index = segment_df[segment_df['bolus_dose'] == closest_bolus['dose']].index[0]
                    if row['timestamp'] - closest_bolus['ts_begin'] < closest_bolus['ts_begin'] - segment_df.at[assigned_index, 'timestamp']:
                        # Reassign the bolus to the new closer timestamp
                        segment_df.at[assigned_index, 'bolus_dose'] = 0  # Remove dose from previously assigned timestamp
                        segment_df.at[i, 'bolus_dose'] = closest_bolus['dose']  # Assign dose to the new closer timestamp
            # else:
            #     print(f"bolus type {bolus['type']} on {bolus['ts']} is too far from closest timestamp in {closest_segment} with a difference of {closest_diff}.")

    return segments

In [10]:
def label_delta_transform(labels_list):
    # label_lower_percentile = -12.75
    # label_upper_percentile = 12.85
    label_lower_percentile = np.percentile(labels_list, 10)
    label_upper_percentile = np.percentile(labels_list, 90)
    transformed_labels = []
    for label in labels_list:
        if label <= label_lower_percentile:
            transformed_labels.append(1)
        elif label_lower_percentile < label < label_upper_percentile:
            trans_label = round((256/(label_upper_percentile - label_lower_percentile))*(label + abs(label_lower_percentile) + 0.05))
            transformed_labels.append(trans_label)
        elif label >= label_upper_percentile:
            transformed_labels.append(256)
    return transformed_labels



def prepare_dataset(segments, ph):
    '''
    ph = 6, 30 minutes ahead
    ph = 12, 60 minutes ahead
    '''
    features_list = []
    labels_list = []
    raw_glu_list = []
    
    # Iterate over each segment
    for segment_name, segment_df in segments.items():
        # Ensure all columns are of numeric type
        # segment_df['carbs'] = pd.to_numeric(segment_df['carbs'], errors='coerce')
        # segment_df['basal_rate'] = pd.to_numeric(segment_df['basal_rate'], errors='coerce')
        # segment_df['bolus_dose'] = pd.to_numeric(segment_df['bolus_dose'], errors='coerce')

        # Fill NaNs that might have been introduced by conversion errors
        segment_df.fillna(0, inplace=True)

        # Maximum index for creating a complete feature set
        print("len of segment_df is ", len(segment_df))
        max_index = len(segment_df) - (history_len + ph)  # Subtracting only 15+ph to ensure i + 15 + ph is within bounds
        
        # Iterate through the data to create feature-label pairs
        for i in range(max_index):
            # Extracting features from index i to i+15
            segment_df = segment_df.reset_index(drop = True)
            features = segment_df.loc[i:i+history_len, ['glucose_value']].values
            # Extracting label for index i+15+ph
            # label = segment_df.loc[i+15+ph, 'glucose_value'] - segment_df.loc[i+15, 'glucose_value']
            
            raw_glu_list.append(segment_df.loc[i+history_len+ph, 'glucose_value'])
            features_list.append(features)
            # labels_list.append(label)
            
    print("len of features_list " + str(len(features_list)))
    # print("len of labels_list " + str(len(labels_list)))
    
    # new_labels_list = label_delta_transform(labels_list)    
    # print("after label transform, the len of label list "+str(len(new_labels_list)))    
    
    return features_list, raw_glu_list

def get_glucose_df(glucose): 
    glucose_dict = {entry[0]['ts']: entry[0]['value'] for entry in glucose}

    # Create the multi-channel database
    g_data = []
    for timestamp in glucose_dict:
        record = {
            'timestamp': timestamp,
            'glucose_value': glucose_dict[timestamp],
            # 'meal_type': None,
            # 'meal_carbs': 0
        }
        
        g_data.append(record)

    # Create DataFrame
    glucose_df = pd.DataFrame(g_data)

    # Convert glucose values to numeric type for analysis
    glucose_df['glucose_value'] = pd.to_numeric(glucose_df['glucose_value'])

    # Calculate percentiles
    lower_percentile = np.percentile(glucose_df['glucose_value'], 2)
    upper_percentile = np.percentile(glucose_df['glucose_value'], 98)

    return glucose_df


In [31]:
# Build the dilate CNN
import torch
import torch.nn as nn
import torch.nn.functional as F

class WaveNetBlock(nn.Module):
    def __init__(self, in_channels, dilation):
        super(WaveNetBlock, self).__init__()
        self.conv1 = nn.Conv1d(in_channels, in_channels, kernel_size=2, dilation=dilation, padding=1+dilation - 2^(dilation-1))
        self.conv2 = nn.Conv1d(in_channels, in_channels, kernel_size=2, dilation=dilation, padding=dilation)
        self.res_conv = nn.Conv1d(in_channels, in_channels, kernel_size=1)
        
    def forward(self, x):
        # print("shape of x: ", x.shape)
        out = F.relu(self.conv1(x))
        # print("shape of first out: ", out.shape)
        out = F.relu(self.conv2(out))
        # print("shape of second out: ", out.shape)
        res = self.res_conv(x)
        # print("shape of res: ", res.shape)
        return out + res

class WaveNet(nn.Module):
    def __init__(self, in_channels, out_channels, num_blocks, dilations):
        super(WaveNet, self).__init__()
        self.initial_conv = nn.Conv1d(in_channels, 32, kernel_size=2, padding=1)
        self.blocks = nn.ModuleList([WaveNetBlock(32, dilation) for dilation in dilations])
        self.final_conv1 = nn.Conv1d(32, 128, kernel_size=2, padding=0)
        self.final_conv2 = nn.Conv1d(128, 256, kernel_size=2, padding=0)
        self.fc1 = nn.Linear(256, 128)
        self.fc2 = nn.Linear(128, out_channels)
        
    def forward(self, x):
        x = F.relu(self.initial_conv(x))
        for block in self.blocks:
            # print("enter the block loop")
            x = block(x)
        x = F.relu(self.final_conv1(x))
        x = F.relu(self.final_conv2(x))
        x = x[:, :, -1]  # Get the last time step
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

input_channels = 1  # Number of features
output_channels = 1  # Predicting a single value (glucose level)
num_blocks = 4  # Number of WaveNet blocks
dilations = [2**i for i in range(num_blocks)]  # Dilation rates: 1, 2, 4, 8

model = WaveNet(input_channels, output_channels, num_blocks, dilations)
print(model)

# Example of how to define the loss and optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0008)


WaveNet(
  (initial_conv): Conv1d(1, 32, kernel_size=(2,), stride=(1,), padding=(1,))
  (blocks): ModuleList(
    (0): WaveNetBlock(
      (conv1): Conv1d(32, 32, kernel_size=(2,), stride=(1,))
      (conv2): Conv1d(32, 32, kernel_size=(2,), stride=(1,), padding=(1,))
      (res_conv): Conv1d(32, 32, kernel_size=(1,), stride=(1,))
    )
    (1): WaveNetBlock(
      (conv1): Conv1d(32, 32, kernel_size=(2,), stride=(1,), dilation=(2,))
      (conv2): Conv1d(32, 32, kernel_size=(2,), stride=(1,), padding=(2,), dilation=(2,))
      (res_conv): Conv1d(32, 32, kernel_size=(1,), stride=(1,))
    )
    (2): WaveNetBlock(
      (conv1): Conv1d(32, 32, kernel_size=(2,), stride=(1,), dilation=(4,))
      (conv2): Conv1d(32, 32, kernel_size=(2,), stride=(1,), padding=(4,), dilation=(4,))
      (res_conv): Conv1d(32, 32, kernel_size=(1,), stride=(1,))
    )
    (3): WaveNetBlock(
      (conv1): Conv1d(32, 32, kernel_size=(2,), stride=(1,), dilation=(8,))
      (conv2): Conv1d(32, 32, kernel_size=(2

# Implementation

In [32]:
subject = pd.read_csv(f"../T1DEXI_processed/1.csv")

In [33]:
subject_to_test_on = '1.csv'
import glob
all_df = []
for file in glob.glob("../T1DEXI_processed/*.csv"):
    glucose = preprocess_t1dexi_cgm(file, False)
    glucose_df = get_glucose_df(glucose)
    segments = segement_data_as_1hour(glucose_df)
    # interpolated_segements = detect_missing_and_spline_interpolate(segments)
    all_df.append(interpolated_segements)

C:\Users\Biratal\AppData\Local\Temp\ipykernel_110652\826782909.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df_cgm['LBDTC'] = pd.to_datetime(new_df_cgm['LBDTC'], errors='coerce')  # Convert 'date' column to datetime if not already
C:\Users\Biratal\AppData\Local\Temp\ipykernel_110652\826782909.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df_cgm.sort_values('LBDTC', inplace=True)  # Sort the DataFrame by the 'date' column


segment_2
before:  2020-05-19 20:56:37
after:  2020-05-19 21:11:37
segment_2
before:  2020-05-20 21:11:38
after:  2020-05-20 21:36:38
segment_2
before:  2020-05-20 22:46:39
after:  2020-05-20 23:06:40


C:\Users\Biratal\AppData\Local\Temp\ipykernel_110652\826782909.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df_cgm['LBDTC'] = pd.to_datetime(new_df_cgm['LBDTC'], errors='coerce')  # Convert 'date' column to datetime if not already
C:\Users\Biratal\AppData\Local\Temp\ipykernel_110652\826782909.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df_cgm.sort_values('LBDTC', inplace=True)  # Sort the DataFrame by the 'date' column


segment_1
before:  2019-12-22 18:19:45
after:  2019-12-22 19:04:46
segment_1
before:  2019-12-25 12:24:51
after:  2019-12-25 13:04:53
segment_2
before:  2019-12-28 15:45:00
after:  2019-12-28 16:20:00
segment_5
before:  2020-01-08 18:00:58
after:  2020-01-08 18:10:58
segment_5
before:  2020-01-08 18:10:58
after:  2020-01-08 18:20:58
segment_5
before:  2020-01-15 11:46:14
after:  2020-01-15 12:06:14
segment_5
before:  2020-01-15 13:31:14
after:  2020-01-15 13:56:14


C:\Users\Biratal\AppData\Local\Temp\ipykernel_110652\826782909.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df_cgm['LBDTC'] = pd.to_datetime(new_df_cgm['LBDTC'], errors='coerce')  # Convert 'date' column to datetime if not already
C:\Users\Biratal\AppData\Local\Temp\ipykernel_110652\826782909.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df_cgm.sort_values('LBDTC', inplace=True)  # Sort the DataFrame by the 'date' column


...

C:\Users\Biratal\AppData\Local\Temp\ipykernel_110652\826782909.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df_cgm['LBDTC'] = pd.to_datetime(new_df_cgm['LBDTC'], errors='coerce')  # Convert 'date' column to datetime if not already
C:\Users\Biratal\AppData\Local\Temp\ipykernel_110652\826782909.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df_cgm.sort_values('LBDTC', inplace=True)  # Sort the DataFrame by the 'date' column


segment_1
before:  2021-03-04 10:47:51
after:  2021-03-04 10:57:50
segment_1
before:  2021-03-04 11:22:50
after:  2021-03-04 11:32:50
segment_1
before:  2021-03-04 16:47:49
after:  2021-03-04 16:57:49
segment_1
before:  2021-03-04 19:22:48
after:  2021-03-04 19:32:48
segment_1
before:  2021-03-04 20:52:48
after:  2021-03-04 21:02:48
segment_1
before:  2021-03-05 05:22:48
after:  2021-03-05 05:32:48
segment_1
before:  2021-03-05 06:17:48
after:  2021-03-05 06:27:47
segment_1
before:  2021-03-05 06:47:46
after:  2021-03-05 06:57:46
segment_1
before:  2021-03-05 07:47:45
after:  2021-03-05 07:57:45
segment_1
before:  2021-03-05 10:42:44
after:  2021-03-05 10:52:45
segment_1
before:  2021-03-05 14:12:45
after:  2021-03-05 14:22:45
segment_1
before:  2021-03-05 17:17:45
after:  2021-03-05 17:27:44
segment_1
before:  2021-03-05 18:07:45
after:  2021-03-05 18:17:44
segment_1
before:  2021-03-05 22:02:43
after:  2021-03-05 22:12:43
segment_1
before:  2021-03-06 06:32:43
after:  2021-03-06 06:4

C:\Users\Biratal\AppData\Local\Temp\ipykernel_110652\826782909.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df_cgm['LBDTC'] = pd.to_datetime(new_df_cgm['LBDTC'], errors='coerce')  # Convert 'date' column to datetime if not already
C:\Users\Biratal\AppData\Local\Temp\ipykernel_110652\826782909.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df_cgm.sort_values('LBDTC', inplace=True)  # Sort the DataFrame by the 'date' column


segment_1
before:  2020-08-05 11:47:39
after:  2020-08-05 11:57:38
segment_1
before:  2020-08-09 08:57:56
after:  2020-08-09 09:07:57
segment_1
before:  2020-08-12 16:03:11
after:  2020-08-12 16:23:11
segment_2
before:  2020-08-18 14:13:40
after:  2020-08-18 14:23:39
segment_2
before:  2020-08-19 23:23:46
after:  2020-08-19 23:43:47
segment_2
before:  2020-08-20 00:53:46
after:  2020-08-20 01:33:46
segment_2
before:  2020-08-20 02:03:46
after:  2020-08-20 02:38:46
segment_3
before:  2020-08-20 04:28:47
after:  2020-08-20 05:18:47
segment_3
before:  2020-08-20 08:53:48
after:  2020-08-20 09:23:48
segment_3
before:  2020-08-20 12:03:49
after:  2020-08-20 12:48:49
segment_3
before:  2020-08-20 13:08:49
after:  2020-08-20 13:48:49
segment_4
before:  2020-08-22 13:18:57
after:  2020-08-22 13:28:57
segment_4
before:  2020-08-27 14:44:22
after:  2020-08-27 14:54:21
segment_4
before:  2020-08-28 09:39:26
after:  2020-08-28 09:49:26
segment_4
before:  2020-08-28 12:49:25
after:  2020-08-28 12:5

C:\Users\Biratal\AppData\Local\Temp\ipykernel_110652\826782909.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df_cgm['LBDTC'] = pd.to_datetime(new_df_cgm['LBDTC'], errors='coerce')  # Convert 'date' column to datetime if not already
C:\Users\Biratal\AppData\Local\Temp\ipykernel_110652\826782909.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df_cgm.sort_values('LBDTC', inplace=True)  # Sort the DataFrame by the 'date' column


In [34]:
# combine eveyrthing in all_df by renaming the segment as i+segment
all_df_combined = {}

for i, segment in enumerate(all_df):
    for segment_name, segment_df in segment.items():
        all_df_combined[str(i) + "_" + segment_name] = segment_df
        


In [35]:
import pickle
ohio_data = pickle.load(open('./pkl_files/BIG_training_data_all_wavenet.pkl', 'rb'))
# get the size of the data
key_size = 0
for key in ohio_data: 
    key_size += len(ohio_data[key])
print(f'{key_size} data points in ohio_data')

key_size = 0
counter = 0
for key in all_df_combined:
    counter += 1    
    key_size += len(all_df_combined[key])
    if counter > 60: 
        break
print(f'{key_size} data points in all_df_combined')


69758 data points in ohio_data
56528 data points in all_df_combined


In [36]:
# only select the first 68 subjects
all_df_combined_subset = {k: all_df_combined[k] for k in list(all_df_combined)[:60]}


In [37]:
len(all_df_combined_subset)

60

In [38]:
bolus_updated_segments = all_df_combined_subset
PH = 6


features_list, raw_glu_list = prepare_dataset(bolus_updated_segments, PH)
# Assuming features_list and raw_glu_list are already defined
features_array = np.array(features_list)
labels_array = np.array(raw_glu_list)

# Step 1: Split into 80% train+val and 20% test
X_temp, X_test, y_temp, y_test = train_test_split(features_array, labels_array, test_size=0.2, shuffle=False)

# Step 2: Split the 80% into 70% train and 10% val (0.7/0.8 = 0.875)
X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=0.125, shuffle=False)

# Convert the splits to torch tensors
X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32)
X_val = torch.tensor(X_val, dtype=torch.float32)
y_val = torch.tensor(y_val, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.float32)

# Create DataLoaders
train_dataset = TensorDataset(X_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=False)

val_dataset = TensorDataset(X_val, y_val)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

test_dataset = TensorDataset(X_test, y_test)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

len of segment_df is  143
len of segment_df is  2745
len of segment_df is  99
len of segment_df is  2856
len of segment_df is  2053
len of segment_df is  2483
len of segment_df is  91
len of segment_df is  107
len of segment_df is  2856
len of segment_df is  2167
len of segment_df is  232
len of segment_df is  545
len of segment_df is  226
len of segment_df is  1720
len of segment_df is  112
len of segment_df is  38
len of segment_df is  101
len of segment_df is  290
len of segment_df is  43
len of segment_df is  157
len of segment_df is  158
len of segment_df is  15
len of segment_df is  158
len of segment_df is  627
len of segment_df is  1158
len of segment_df is  71
len of segment_df is  27
len of segment_df is  259
len of segment_df is  396
len of segment_df is  994
len of segment_df is  70
len of segment_df is  109
len of segment_df is  1187
len of segment_df is  296
len of segment_df is  73
len of segment_df is  1025
len of segment_df is  603
len of segment_df is  152
len of segm

In [39]:

# Training Loop
num_epochs = 100
for epoch in range(num_epochs):
    model.train()
    for inputs, targets in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs.permute(0, 2, 1))  # Permute to match (batch, channels, seq_len)
        # use squeeze
        outputs = outputs.squeeze()
        targets = targets.squeeze()
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

    model.eval()
    val_loss = 0
    with torch.no_grad():
        for inputs, targets in val_loader:
            outputs = model(inputs.permute(0, 2, 1))  # Permute to match (batch, channels, seq_len)
            outputs = outputs.squeeze()
            targets = targets.squeeze()

            loss = criterion(outputs, targets)
            val_loss += loss.item() 
    
    print(f'Epoch {epoch+1}, Validation Loss: {val_loss / len(val_loader)}')

Epoch 1, Validation Loss: 341.95297687781783
Epoch 2, Validation Loss: 331.9453991758609
Epoch 3, Validation Loss: 385.5287616752579
Epoch 4, Validation Loss: 404.67371295312205
Epoch 5, Validation Loss: 386.6629343089943
Epoch 6, Validation Loss: 351.86400499172555
Epoch 7, Validation Loss: 340.6528369652297
Epoch 8, Validation Loss: 349.89051654381666
Epoch 9, Validation Loss: 359.8715776911753
Epoch 10, Validation Loss: 365.2281039118053
Epoch 11, Validation Loss: 369.6546464565985
Epoch 12, Validation Loss: 448.91213148653867
Epoch 13, Validation Loss: 501.0332405866977
Epoch 14, Validation Loss: 519.8676311926928
Epoch 15, Validation Loss: 549.618913022344
Epoch 16, Validation Loss: 559.8014026573318
Epoch 17, Validation Loss: 374.73011409736677
Epoch 18, Validation Loss: 403.0161593317272
Epoch 19, Validation Loss: 352.7062080063506
Epoch 20, Validation Loss: 375.101546007716
Epoch 21, Validation Loss: 396.73197187777765
Epoch 22, Validation Loss: 350.3141584339256
Epoch 23, Vali

In [20]:
# save the model 
torch.save(model.state_dict(), f'./models/glucnet_t1dexi_{PH}_wavenet.pt')

In [41]:
# load the model 
print('no')
# model_dir = f'./models/glucnet_t1dexi_6_wavenet.pt'

# model.load_state_dict(torch.load(model_dir))

no


In [42]:
# Calculate RMSE on test set
model.eval()
predictions = []
actuals = []
with torch.no_grad():
    for inputs, targets in test_loader:
        outputs = model(inputs.permute(0, 2, 1))
        predictions.append(outputs)
        actuals.append(targets)

predictions = torch.cat(predictions).cpu().numpy()
actuals = torch.cat(actuals).cpu().numpy()


rmse = np.sqrt(mean_squared_error(actuals,predictions))
print(f'RMSE on test set: {rmse}')

RMSE on test set: 18.535480499267578


# Implement on the group